In [ ]:
import geopandas as gpd # type: ignore 
# why does vscode flag geopandas, pandas as not being found?
import pandas as pd # type: ignore 

In [71]:
from pathlib import Path
from typing import List, Dict

In [ ]:
df_dogs = pd.read_csv(Path.cwd().joinpath('data').joinpath('seattle_dogs_single_breed.csv'))


In [86]:
gdf = gpd.read_file(Path.cwd().joinpath('data').joinpath('clean_seattle_zipcodes.geojson'))

In [19]:
df_dogs.head()

,zip,breed,count
0,98101,airedale terrier,1
1,98101,akita,1
2,98101,american eskimo,1
3,98101,australian shepherd,4
4,98101,basenji,1


In [ ]:
gdf_raw.head()

,zipcode,geometry
0,98422,"MULTIPOLYGON (((-122.39386 47.30380, -122.3935..."
1,98321,"MULTIPOLYGON (((-122.11344 47.21775, -122.1138..."
2,98323,"MULTIPOLYGON (((-121.93754 47.13888, -121.9374..."
3,98354,"MULTIPOLYGON (((-122.33425 47.25737, -122.3342..."
4,98372,"MULTIPOLYGON (((-122.28811 47.25720, -122.2879..."


In [ ]:
# gdf_raw.zipcode.isna().sum() # 0
df_dogs.zip.isna().sum() # also 0

0

Do our transformations on the source data, so we're only joining zips in a 1-1 relationship.
2 needs:
- top breeds in each zip code
- able to extract number of a specific breed in each zip code

We could store these as a dictionary per each zip code, of all breed counts in the zip code?
Then call first five OR look up specific breed.

Oh yeah, and, let's also generate the top breeds city-wide & display that as a list.

In [94]:
print(df_dogs[['breed', 'count']].groupby('breed').agg('sum'))

                             count
breed                             
affenpinscher                    3
afghan hound                     2
ainu                             1
airedale terrier                24
akbash                           2
...                            ...
wire fox terrier                12
wire haired vizsla               3
wirehaired pointing griffon     29
xoloitzcuintli                  37
yorkshire terrier              111

[252 rows x 1 columns]


In [97]:
df_dogs[['breed', 'count']].groupby('breed').agg('sum').reset_index().to_dict()

{'breed': {0: 'affenpinscher',
  1: 'afghan hound',
  2: 'ainu',
  3: 'airedale terrier',
  4: 'akbash',
  5: 'akita',
  6: 'alaskan husky',
  7: 'alaskan klee kai',
  8: 'alaskan malamute',
  9: 'american blue heeler',
  10: 'american bulldog',
  11: 'american bullnese',
  12: 'american cocker spaniel',
  13: 'american crested sand terrier',
  14: 'american english coonhound',
  15: 'american eskimo',
  16: 'american foxhound',
  17: 'american hairless terrier',
  18: 'american indian dog',
  19: 'american pit bull terrier',
  20: 'american staffordshire terrier',
  21: 'american water spaniel',
  22: 'american white shepherd',
  23: 'anatolian shepherd',
  24: 'appenzeller sennenhunde',
  25: 'argentine dogo',
  26: 'australian cattle dog',
  27: 'australian kelpie',
  28: 'australian shepherd',
  29: 'australian terrier',
  30: 'barbet',
  31: 'basenji',
  32: 'basset hound',
  33: 'beagle',
  34: 'bearded collie',
  35: 'beauceron',
  36: 'bedlington terrier',
  37: 'belgian malino

In [99]:
df_citywide = df_dogs[['breed', 'count']].groupby('breed').agg('sum').reset_index()# df_dogs[df_dogs.zip==98103][['breed', 'count']].to_dict(orient='index')

In [121]:
top_breeds_str = """City-wide Top Breeds (Breed, count):\n"""

for (breedname, count) in df_citywide.sort_values(by='count', ascending=False).values:
    top_breeds_str+=f"{breedname}:\t{count}\n"

print(top_breeds_str)

City-wide Top Breeds (Breed, count):
labrador retriever:	1025
golden retriever:	716
australian shepherd:	282
german shepherd:	277
pembroke welsh corgi:	266
standard poodle:	235
havanese:	230
french bulldog:	222
short coat chihuahua:	217
miniature australian shepherd:	190
siberian husky:	175
miniature poodle:	173
cavalier king charles spaniel:	158
shiba inu:	142
shih tzu:	142
boston terrier:	138
border collie:	138
pug:	133
retriever:	131
miniature schnauzer:	126
yorkshire terrier:	111
beagle:	110
american pit bull terrier:	104
boxer:	103
greyhound:	100
pomeranian:	94
terrier:	92
australian cattle dog:	91
miniature smooth haired dachshund:	89
portuguese water dog:	89
english springer spaniel:	88
bernese mountain dog:	84
miniature long haired dachshund:	82
smooth haired vizsla:	78
german shorthaired pointer:	76
west highland white terrier:	74
toy poodle:	68
lagotto romagnolo:	60
maltese:	59
great dane:	57
samoyed:	56
doberman pinscher:	53
soft coated wheaten terrier:	52
american cocker sp

In [119]:
top_breeds_str

'City-wide Top Breeds (Breed, count):\\nlabrador retriever:\\t1025\\ngolden retriever:\\t716\\naustralian shepherd:\\t282\\ngerman shepherd:\\t277\\npembroke welsh corgi:\\t266\\nstandard poodle:\\t235\\nhavanese:\\t230\\nfrench bulldog:\\t222\\nshort coat chihuahua:\\t217\\nminiature australian shepherd:\\t190\\nsiberian husky:\\t175\\nminiature poodle:\\t173\\ncavalier king charles spaniel:\\t158\\nshiba inu:\\t142\\nshih tzu:\\t142\\nboston terrier:\\t138\\nborder collie:\\t138\\npug:\\t133\\nretriever:\\t131\\nminiature schnauzer:\\t126\\nyorkshire terrier:\\t111\\nbeagle:\\t110\\namerican pit bull terrier:\\t104\\nboxer:\\t103\\ngreyhound:\\t100\\npomeranian:\\t94\\nterrier:\\t92\\naustralian cattle dog:\\t91\\nminiature smooth haired dachshund:\\t89\\nportuguese water dog:\\t89\\nenglish springer spaniel:\\t88\\nbernese mountain dog:\\t84\\nminiature long haired dachshund:\\t82\\nsmooth haired vizsla:\\t78\\ngerman shorthaired pointer:\\t76\\nwest highland white terrier:\\t74\\nt

In [ ]:
# df_dogs[df_dogs.zip==98103][['breed', 'count']].to_dict(orient='index')
grouped = df_dogs.groupby('zip')


In [ ]:

breed_counts_by_zip = {}
# dict of a dict:  {zipcode:{breed:count})

for i, (name, df) in enumerate(grouped):

    temp_dict = {}
    # note, there may be some null /empty dicts!
    
    # print(name)
    # print(df.columns) # 'zip', 'breed', 'count'
    # print(df[['breed', 'count']].to_dict(orient='records')) # [{'breed': 'australian shepherd', 'count': 4}]
    
    breed_count: List[Dict] = df[['breed', 'count']].to_dict(orient='records') # list of breeds

    # Let's sort the dictionaries by count, instead of 

    for elem_dict in breed_count:
        # e.g. unpack {'breed': 'australian shepherd', 'count': 4}
        # to {'australian shepherd':4}
        # breed_counts_by_zip[name].update({elem_dict.get('breed'): elem_dict.get('count')})
        temp_dict.update({elem_dict.get('breed'): elem_dict.get('count')})
        
    # Let's sort the dictionaries by count, instead of alphabetically
    breed_counts_by_zip[name] = dict(sorted(temp_dict.items(), key=lambda item: item[1], reverse=True))
    
    

In [81]:
breed_counts_by_zip.keys()

dict_keys([98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98125, 98126, 98127, 98132, 98133, 98134, 98136, 98139, 98144, 98146, 98147, 98155, 98166, 98168, 98177, 98178, 98188, 98195, 98198, 98199, 98202, 98204, 98215, 98258, 98260, 98332, 98367, 98375, 98391, 98405, 98502, 98507, 98937, 99103, 99166])

In [82]:
breed_counts_by_zip[98104]

{'short coat chihuahua': 10,
 'american cocker spaniel': 3,
 'miniature long haired dachshund': 3,
 'american pit bull terrier': 2,
 'border collie': 2,
 'doberman pinscher': 2,
 'havanese': 2,
 'pembroke welsh corgi': 2,
 'pomeranian': 2,
 'shiba inu': 2,
 'standard poodle': 2,
 'alaskan husky': 1,
 'alaskan malamute': 1,
 'australian shepherd': 1,
 'bichon frise': 1,
 'boxer': 1,
 'cavalier king charles spaniel': 1,
 'chinese crested': 1,
 'chinese shar-pei': 1,
 'english bulldog': 1,
 'french bulldog': 1,
 'german shepherd': 1,
 'golden retriever': 1,
 'irish setter': 1,
 'irish terrier': 1,
 'italian greyhound': 1,
 'maltese': 1,
 'miniature australian shepherd': 1,
 'miniature pinscher': 1,
 'miniature poodle': 1,
 'miniature schnauzer': 1,
 'miniature smooth haired dachshund': 1,
 'pit bull terrier': 1,
 'pug': 1,
 'rat terrier': 1,
 'siberian husky': 1,
 'small (under 24 lbs fully grown) mixed breed': 1,
 'smooth fox terrier': 1,
 'standard smooth haired dachshund': 1,
 'xoloitz

In [ ]:
sorted(temp.items(), key=lambda item: item[1], reverse=True)

[('french bulldog', 7),
 ('shih tzu', 7),
 ('golden retriever', 6),
 ('short coat chihuahua', 6),
 ('labrador retriever', 5),
 ('miniature poodle', 5),
 ('australian shepherd', 4),
 ('cavalier king charles spaniel', 4),
 ('havanese', 4),
 ('pembroke welsh corgi', 4),
 ('pomeranian', 4),
 ('standard poodle', 4),
 ('yorkshire terrier', 4),
 ('siberian husky', 3),
 ('basset hound', 2),
 ('boxer', 2),
 ('bull terrier', 2),
 ('german shepherd', 2),
 ('greyhound', 2),
 ('miniature australian shepherd', 2),
 ('miniature pinscher', 2),
 ('rough collie', 2),
 ('shiba inu', 2),
 ('staffordshire bull terrier', 2),
 ('toy poodle', 2),
 ('west highland white terrier', 2),
 ('airedale terrier', 1),
 ('akita', 1),
 ('american eskimo', 1),
 ('basenji', 1),
 ('belgian sheepdog', 1),
 ('bichon frise', 1),
 ('border collie', 1),
 ('border terrier', 1),
 ('boston terrier', 1),
 ('bulldog', 1),
 ('bullmastiff', 1),
 ('long coat chihuahua', 1),
 ('lowchen', 1),
 ('maltese', 1),
 ('miniature long haired dach

In [ ]:
dict(sorted(temp.items(), key=lambda item: item[1], reverse=True)) # yay!

{'french bulldog': 7,
 'shih tzu': 7,
 'golden retriever': 6,
 'short coat chihuahua': 6,
 'labrador retriever': 5,
 'miniature poodle': 5,
 'australian shepherd': 4,
 'cavalier king charles spaniel': 4,
 'havanese': 4,
 'pembroke welsh corgi': 4,
 'pomeranian': 4,
 'standard poodle': 4,
 'yorkshire terrier': 4,
 'siberian husky': 3,
 'basset hound': 2,
 'boxer': 2,
 'bull terrier': 2,
 'german shepherd': 2,
 'greyhound': 2,
 'miniature australian shepherd': 2,
 'miniature pinscher': 2,
 'rough collie': 2,
 'shiba inu': 2,
 'staffordshire bull terrier': 2,
 'toy poodle': 2,
 'west highland white terrier': 2,
 'airedale terrier': 1,
 'akita': 1,
 'american eskimo': 1,
 'basenji': 1,
 'belgian sheepdog': 1,
 'bichon frise': 1,
 'border collie': 1,
 'border terrier': 1,
 'boston terrier': 1,
 'bulldog': 1,
 'bullmastiff': 1,
 'long coat chihuahua': 1,
 'lowchen': 1,
 'maltese': 1,
 'miniature long haired dachshund': 1,
 'miniature schnauzer': 1,
 'miniature smooth haired dachshund': 1,
 '

In [24]:
df_dogs.tail(10)

,zip,breed,count
2441,98375,english bulldog,1
2442,98391,siberian husky,1
2443,98405,french bulldog,1
2444,98502,labrador retriever,1
2445,98507,smooth haired vizsla,1
2446,98507,toy poodle,1
2447,98507,yorkshire terrier,1
2448,98937,shih tzu,1
2449,99103,standard poodle,2
2450,99166,american blue heeler,1


In [ ]:
df_dogs[df_dogs.zip==98105].breed.nunique() # 102

102

In [ ]:
# df_dogs.groupby(by='zip', sort=True).value_counts()
df_dogs.groupby(by='zip')

zip    breed                                          count
98101  airedale terrier                               1        1
98125  beagle                                         6        1
       american cocker spaniel                        6        1
       american eskimo                                3        1
       american pit bull terrier                      1        1
                                                              ..
98112  maltese                                        1        1
       medium (up to 44 lbs fully grown) mixed breed  1        1
       miniature australian shepherd                  5        1
       miniature long haired dachshund                2        1
99166  american blue heeler                           1        1
Length: 2451, dtype: int64

In [87]:
gdf.head()

,id,zipcode,geometry
0,98422,98422,"MULTIPOLYGON (((-122.39386 47.30380, -122.3935..."
1,98321,98321,"MULTIPOLYGON (((-122.11344 47.21775, -122.1138..."
2,98323,98323,"MULTIPOLYGON (((-121.93754 47.13888, -121.9374..."
3,98354,98354,"MULTIPOLYGON (((-122.33425 47.25737, -122.3342..."
4,98372,98372,"MULTIPOLYGON (((-122.28811 47.25720, -122.2879..."


In [ ]:
# gdf = pd.merge(gdf_raw, df_dogs, how='inner',
                # left_on='')

In [123]:
import json

geo_filepath = Path.cwd().joinpath('data').joinpath('clean_seattle_zipcodes.geojson')
gdf = gpd.read_file(geo_filepath)
gdf.zipcode = gdf.zipcode.astype('int64') # for merging later

top_breeds_by_zip_df = pd.read_csv(Path.cwd().joinpath('data').joinpath('top_5_breeds_by_zip.csv'))
gdf = gdf.merge(top_breeds_by_zip_df, how='inner', left_on='zipcode', right_on='zip')
gdf = gdf[['zip', 'geometry', 'top_5_breeds']]
with open(geo_filepath, 'r') as f:
   geodata = json.load(f)

df = pd.read_csv(Path.cwd().joinpath('data').joinpath('seattle_dogs_single_breed.csv'))
df.rename({'zip':'zipcode'}, inplace=True)

In [128]:
raw=df[df['breed']=='french bulldog'][['zip', 'count']].to_dict('records')
choro_data = {}
for elem in raw:
    choro_data[str(elem['zip'])] = elem['count']

In [129]:
for zip in gdf.zip.unique():
    if str(zip) not in choro_data.keys():
        choro_data[str(zip)] = 0

In [136]:
geodata['features'][0]['properties']

{'zipcode': '98422'}

In [139]:
len(choro_data.keys())

35

In [ ]:
gdf.zip.nunique() # there's one missing?

34

In [148]:
choro_data['98422']=0

In [149]:
choro_data['98422'] # does not exist...

0

In [ ]:
gdf.zip.unique()

array([98391, 98136, 98144, 98146, 98155, 98101, 98166, 98168, 98177,
       98178, 98188, 98195, 98198, 98199, 98102, 98103, 98104, 98105,
       98106, 98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118,
       98119, 98121, 98122, 98125, 98126, 98133, 98134])

In [152]:
choro_zips = sorted([int(k) for k in choro_data.keys()])


In [153]:
for ch in choro_zips:
    if ch not in gdf.zip.unique():
        print(f"'{ch}', ")

'98405', 
'98422', 
